# Exploration of Data mining the PPDB database

http://sitem.herts.ac.uk/aeru/ppdb/en/atoz.htm

In [1]:
from bs4 import BeautifulSoup
from IPython.display import display
import requests
from datetime import date, timedelta, datetime
import time
from datetime import datetime
import pandas as pd
import os
import winsound
import numpy as np

pd.set_option("display.max_columns", None)

In [2]:
%load_ext autoreload
%autoreload 2


def try_get_column_value_else_nan(dfs: list, col_name: str):
    """Get the value of the first row with the column set in the arguments. If not column doens't exist, return NaN.

    When webscraping tables you sometimes stumble upon a table that's missing a certain column.
    To avoid KeyErrors breaking your scrape loop, this function catches the KeyError and returns a NaN instead.
    """
    output = np.nan
    for df in dfs:
        try:
            output = df.loc[0, col_name]
        except KeyError:
            continue

    return output

## Scraping

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/115.0"
}

### PPDB database
http://sitem.herts.ac.uk/aeru/ppdb/en/atoz.htm

In [4]:
feature_list = [
    "Substance groups",
    "Mode of action",
    "CAS name",
    "CAS RN",
    "EC number",
    "IUPAC name",
    "EC Regulation 1107/2009 status",
    "Dossier rapporteur/co-rapporteur",
    "Date EC 1107/2009 inclusion expires",
    "EU Candidate for substitution (CfS)",
    "Listed in EU database",
]

special_list = ["Threshold of Toxicological Concern (Cramer Class)",
                "Solubility - In water at 20 °C (mg l⁻¹)",
                "Octanol-water partition coefficient at pH 7, 20 °C"]

In [5]:
url_database = "http://sitem.herts.ac.uk/aeru/ppdb/en/atoz.htm"

file_name = "Pesticide_Database_PPDB"
file_location = "../data/raw/"
ENABLE_FINISHED_BEEP = True

date_today = datetime.today().strftime("%Y%m%d")

# Load scraped database if it exists already
if os.path.isfile(f"{file_location}{file_name}"):
    df_pesticide = pd.read_excel(f"{file_location}{date_today}_{file_name}.xlsx")
    print("Dataframe loaded")
# Else: scrape database
else:
    # Inialisation
    pesticides = {} # library with each element being a single pesticide with its respective properties
    i_pest = 0  # index of pesticide, used to move scraped info into the pesticides library
    
    time_start = time.time() # start time of the scraping => to display total scrape time
    # Play beep after scrape finished:
    duration = 750  # milliseconds
    freq = 4 * 440  # Hz

    # Beautiful soup the database
    response = requests.get(url_database, headers=headers)
    # read encoding from webpage to ensure special characters are scraped correctly, if missing: None
    encoding = (
        response.encoding
        if "charset" in response.headers.get("content-type", "").lower()
        else None
    )
    soup = BeautifulSoup(response.content, "html.parser", from_encoding=encoding)

    # Pesticides are alphabetically grouped: loop over all 'p' tags.
    # Note: there are 53 'p' tags, but the even 'p' tag contains either the alphabetical (/numerical) header
    # or an email address (the last group). These are detected and automatically skipped. 
    groups = soup.find_all("p")
    n_groups = len(groups)

    # Initialise progress display
    disp1 = display(f"PPDB Database Scrape has started:", display_id=True)
    disp2 = display(f"Elapsed Time:", display_id=True)

    # Loop over all groups
    for i_group, group in enumerate(groups):  # skip first group as this contains the header
        # group = groups[i_group]
        # Get pesticide items from this group
        items = group.select("a")
        n_item = len(items)

        # Loop over all items in one group
        for i_item, item in enumerate(items):
            # Extract name and url from the item
            name_pest = item.text
            if (
                name_pest == "" or name_pest == "aeru@herts.ac.uk\n"
            ):  # if name_pest is empty, then it hit a new alphabetical header ('A', 'B', etc) => skip this item
                continue
            url_pest = url_database.replace("atoz.htm", "") + item.get("href")
            group_letter = name_pest[0].upper()
            
            # use pandas automatically extract tables from the webpage
            dfs = pd.read_html(url_pest)
            dfs_clean = dfs.copy()
            # Clean up df's as they are detected in long format with the wrong headers => used for normal value feature search
            for i, df in enumerate(dfs):
                # Transpose
                df = df.transpose()
                # Set first row as header
                df.columns = df.iloc[0, :]
                # Remove first row and reset index
                dfs_clean[i] = df[1:].reset_index()

            # Store pesticide info in dictionary
            # Not all fields are always present: check them one by one and if missing => NaN
            pesticides[i_pest] = {"Pesticide Common Name": name_pest, "url": url_pest}
            for feature in feature_list:
                pesticides[i_pest][feature] = try_get_column_value_else_nan(
                    dfs=dfs_clean, col_name=feature
                )

            # Some fields are burried within tables within tables, there require special actions to extract
            dfs_special = dfs_clean.copy()
            dfs_special = [df[1:].reset_index() for df in dfs_special]
            for feature in special_list:
                if feature == "Octanol-water partition coefficient at pH 7, 20 °C":
                    # This feature has two units, P and Log P, so it returns a series instead of a single value => take 2nd element
                    pesticides[i_pest][feature+" - Log P"] = try_get_column_value_else_nan(dfs=dfs_special, col_name=feature).iloc[1]
                else:
                    pesticides[i_pest][feature] = try_get_column_value_else_nan(dfs=dfs_special, col_name=feature)

            # Health issues are stored in a table within a table within a table, so this requires extra actions
            # First detect the right table (as the index is not alway the same), in the raw (non cleaned) dfs
            for df in dfs:
                if df.iloc[0, 0] == "Specific human health issues":
                    # The structure of the table is header , header, header; value, value, value; header, header, header;...
                    for row in range(1, 9, 2):
                        for column in range(0, 3):
                            if not (
                                (row >= 7) and (column == 2)
                            ):  # this table element is empty
                                pesticides[i_pest][df.iloc[row, column]] = df.iloc[
                                    row + 1, column
                                ]

                    # General human health issues again has a different location in the same table:
                    pesticides[i_pest][df.iloc[9, 0]] = df.iloc[9, 2]
                    break

            # Lastly Approved for use under EC 1107/20009 has to be picked from seperate sections of a scraped table
            # First find the correct table among the raw (non cleaned) dfs:
            for df in dfs:
                if df.iloc[0, 0] == "EC Regulation 1107/2009 status":
                    for row in range(6, 11, 2):
                        for column in range(0, 9):
                            # Append country name to generic header. Note BEBelgium is the scraped string, so only take the full countryname
                            feature = (
                                "Approved for use (✓) under EC 1107/2009 - "
                                + df.iloc[row, column][2::]
                            )
                            value = df.iloc[row + 1, column]
                            if value == "&nbsp":  # blank values are detected as bs
                                value = "✗"
                            pesticides[i_pest][feature] = value
                    # EEA countries are in a different section
                    for column in range(0, 2):
                        row = 13
                        # Append country name to generic header. Note BEBelgium is the scraped string, so only take the full countryname
                        feature = (
                            "Approved for use (✓) under EC 1107/2009 - "
                            + df.iloc[row, column][2::]
                        )
                        value = df.iloc[row + 1, column]
                        if value == "&nbsp":
                            value = "✗"
                        pesticides[i_pest][feature] = value
                    break  # break this loop as soon as we've found the right table

            # update pesticide index for the next iteration
            i_pest = i_pest + 1

            # Update progress and time displays
            time_elapsed = time.time() - time_start
            disp1.update(f"Group: {group_letter} - Item: {i_item+1}/{n_item} - Pesticides scraped: {i_pest}")
            disp2.update(f"Time elapsed: {round(time_elapsed/60, 1)} min")
                
            time.sleep(0.3)  # wait between each scrape to avoid ban
            del (
                df,
                dfs,
                dfs_clean,
            )  # to make sure no info from the previous iteration leaks into the next

    # Dict to dataframe
    df_pesticide = pd.DataFrame.from_dict(pesticides, "index")

    # Export
    #   Store dataframe so you don't have to scrape every time
    df_pesticide.to_excel(f"{file_location}{date_today}_{file_name}.xlsx", index=False)
    print("Dataframe saved")

    # Scrape timer
    time_elapsed = time.time() - time_start
    disp2.update(
        f"Total Time: {round(time_elapsed/60, 1)} min. (avg. {round(time_elapsed/i_pest, 1)} sec. per pesticide)"
    )

    if ENABLE_FINISHED_BEEP:
        winsound.Beep(freq, duration)

# Print df info:
print(df_pesticide.shape)
df_pesticide.head()

'Group: Z - Item: 8/8 - Pesticides scraped: 3863'

'Total Time: 39.4 min. (avg. 0.6 sec. per pesticide)'

Dataframe saved
(3863, 57)


,Pesticide Common Name,url,Substance groups,Mode of action,CAS name,CAS RN,EC number,IUPAC name,EC Regulation 1107/2009 status,Dossier rapporteur/co-rapporteur,Date EC 1107/2009 inclusion expires,EU Candidate for substitution (CfS),Listed in EU database,Threshold of Toxicological Concern (Cramer Class),Solubility - In water at 20 °C (mg l⁻¹),"Octanol-water partition coefficient at pH 7, 20 °C - Log P",Carcinogen,Genotoxic,Endocrine disruptor,Reproduction / development effects,Acetyl cholinesterase inhibitor,Neurotoxicant,Respiratory tract irritant,Skin irritant,Skin sensitiser,Eye irritant,Phototoxicant,General human health issues,Approved for use (✓) under EC 1107/2009 - Austria,Approved for use (✓) under EC 1107/2009 - Belgium,Approved for use (✓) under EC 1107/2009 - Bulgaria,Approved for use (✓) under EC 1107/2009 - Cyprus,Approved for use (✓) under EC 1107/2009 - Czech Republic,Approved for use (✓) under EC 1107/2009 - Germany,Approved for use (✓) under EC 1107/2009 - Denmark,Approved for use (✓) under EC 1107/2009 - Estonia,Approved for use (✓) under EC 1107/2009 - Greece,Approved for use (✓) under EC 1107/2009 - Spain,Approved for use (✓) under EC 1107/2009 - Finland,Approved for use (✓) under EC 1107/2009 - France,Approved for use (✓) under EC 1107/2009 - Croatia,Approved for use (✓) under EC 1107/2009 - Hungary,Approved for use (✓) under EC 1107/2009 - Ireland,Approved for use (✓) under EC 1107/2009 - Italy,Approved for use (✓) under EC 1107/2009 - Lithuania,Approved for use (✓) under EC 1107/2009 - Luxembourg,Approved for use (✓) under EC 1107/2009 - Latvia,Approved for use (✓) under EC 1107/2009 - Malta,Approved for use (✓) under EC 1107/2009 - Netherlands,Approved for use (✓) under EC 1107/2009 - Poland,Approved for use (✓) under EC 1107/2009 - Portugal,Approved for use (✓) under EC 1107/2009 - Romania,Approved for use (✓) under EC 1107/2009 - Sweden,Approved for use (✓) under EC 1107/2009 - Slovenia,Approved for use (✓) under EC 1107/2009 - Slovakia,Approved for use (✓) under EC 1107/2009 - Iceland,Approved for use (✓) under EC 1107/2009 - Norway
0,(4-chlorophenoxy)acetic acid,http://sitem.herts.ac.uk/aeru/ppdb/en/Reports/...,Auxin PGR,Auxin-transport inhibitor. Absorbed by tissue ...,(4-chlorophenoxy)acetic acid,122-88-3,204-581-3,4-chlorophenoxyacetic acid,Not approved,Not applicable,Expired,Not applicable,Yes,High (class III),957,2.25,"XNo, known not to cause a problem",A0 A = Chromosome aberration (EFSA database) 0...,No data found,"?Possibly, status not identified","XNo, known not to cause a problem","XNo, known not to cause a problem",No data found,"✓Yes, known to cause a problem",No data found,"✓Yes, known to cause a problem",No data found,Potential irritant Potential kidney toxicant,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗
1,"(E)-2-(2-(2-(2,3-dichlorophenylamino)-6-triflu...",http://sitem.herts.ac.uk/aeru/ppdb/en/Reports/...,Strobilurin insecticide,-,-,-,-,"(E)-2-(2-(2-(2,3-dichlorophenylamino)-6-triflu...",Not approved,Not applicable,Not applicable,Not applicable,No,High (class III),-,-,No data found,A0 A = Chromosome aberration (EFSA database) 0...,No data found,No data found,No data found,No data found,No data found,"XNo, known not to cause a problem",No data found,"✓Yes, known to cause a problem",No data found,No further information available,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗
2,(R)-flufiprole,http://sitem.herts.ac.uk/aeru/ppdb/en/Reports/...,Phenylpyrazole insecticide,-,-,-,No data found,"1-[2,6-dichloro-4-(trifluoromethyl)phenyl]-5-(...",Not approved,Not applicable,Not applicable,Not applicable,No,High (class III),-,-,No data found,A0 A = Chromosome aberration (EFSA database) 0...,No data found,No data found,No data found,No data found,No data found,No data found,No data found,No data found,No data found,No information available,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗
3,(R)-flutriafol,http://sitem.herts.ac.uk/aeru/ppdb/en/Reports/...,Conazole 

### Save requirements.txt

In [7]:
!pipreqsnb --use-local --encoding=iso-8859-1 --force ..
# --use-local ONLY local package info instead of querying PyPI
# --encoding=iso-8859-1 to avoid encoding errors related to utf-8
# --force overwrites current file
# .. saves it in parent folder

pipreqs  --use-local --force --encoding iso-8859-1 ..


INFO: Successfully saved requirements file in ..\requirements.txt
